Installing the SimFin  and yFinance API and importing the necessary libraries.

In [0]:
!pip install simfin
!pip install yfinance
import simfin as sf
from simfin.names import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
np.load.__defaults__=(None, True, True, 'ASCII')
sf.set_api_key('free')
import yfinance as yf

Loading the datasets which will be transformed and printing out their shapes.

In [0]:
sf.set_data_dir('/content/drive/My Drive/Data Sets')

cashflow_df = sf.load_cashflow(variant='quarterly', market='us')
balance_Df = sf.load_balance(variant= 'quarterly', market = 'us')
income_df = sf.load_income(variant='quarterly', market='us')

print(cashflow_df.shape)
print(balance_Df.shape)
print(income_df.shape)

Removing duplicate columns and combining the three datasets into a single one. The dataset is can be saved as a .csv file for easy accesss. Uncomment the code for saving and loading the ready dataset. 

In [0]:
cashflow_df = cashflow_df.drop(columns=['SimFinId', 'Currency','Fiscal Year', 'Fiscal Period', 'Publish Date', 'Shares (Basic)', 'Shares (Diluted)', 'Depreciation & Amortization'])
balance_Df = balance_Df.drop(columns=['SimFinId', 'Currency','Fiscal Year', 'Fiscal Period', 'Publish Date', 'Shares (Basic)', 'Shares (Diluted)'])
income_df = income_df.join(cashflow_df)
big_data = income_df.join(balance_Df)
#big_data.reset_index(inplace=True)
#big_data.to_csv('/content/drive/My Drive/Data Sets/combined_fundamental.csv')
#big_data = pd.read_csv('/content/drive/My Drive/Data Sets/combined_fundamental.csv')
#big_data.set_index(['Ticker', 'Report Date'], inplace=True)
#big_data.drop(columns='Unnamed: 0', inplace=True)
big_data

The yFinance API is used to get the daily pricing data of the SP 500 index and the companies in the fundamental data set. The dates of the reports are cross-referenced with the price for the same day of both the index and the company. We check if the report release date was during the weekend or on a holiday and attempt to get the last available price if they are. If we are unable to find a price we move on to the next company. Takes approximately 30 minutes to run.

In [0]:
# Get price data for the SP 500
ticker_index = yf.Ticker("^GSPC")
history_index = ticker_index.history(period='max', interval='1d')
history_index.drop(columns= ['Dividends', 'Stock Splits'])
time.sleep(1)

ticker = ""

# Iterate through all companies
for index, row in big_data.iterrows():
  try:
    date = datetime.strptime(str(index[1]), '%Y-%m-%d').date()
  except:
    date = datetime.strptime(str(index[1]), '%Y-%m-%d %H:%M:%S').date()

  # Check if we have reached a different company and update price data accordingly
  if ticker != str(index[0]):
    print("Old ticker", ticker)
    ticker = str(index[0])
    print("New ticker", ticker)
    yh_tick = yf.Ticker(ticker)
    history_stock = yh_tick.history(period='max', interval = '1d')
    if history_stock.empty:
      time.sleep(1)
      continue
    time.sleep(1)

  # Check if the report has been released on Saturday and attempt to find price data
  if date.weekday() == 5:
    try:
      big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=1)), 'Close']
      big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=1)), 'Close']
    except:
      print("Saturday - 2")
      try:
        big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=2)), 'Close']
        big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=2)), 'Close']
      except:
        print('Saturday - 4')
        try:
          big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=4)), 'Close']
          big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=4)), 'Close']
        except:
          print("Skipped")
          continue
  
  # Check if the report has been released on Sunday and attempt to find price data 
  elif date.weekday() == 6:
    try:
      big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=2)), 'Close']
      big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=2)), 'Close']
    except:
      print("Sunday - 2")
      try:
        big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=3)), 'Close']
        big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=3)), 'Close']
      except:
        print('Sunday - 4')
        try:
          big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=4)), 'Close']
          big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=4)), 'Close']
        except:
          print("Skipped")
          continue

  # If the report has been released on a weekday attempt to get the price.
  else:
    try:
      big_data.at[(index), 'Stock Price'] = history_stock.at[str(date), 'Close']
      big_data.at[(index), 'Index Price'] = history_index.at[str(date), 'Close']
    except:
      print('Work day - 2')
      try:
        big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=2)), 'Close']
        big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=2)), 'Close']
      except:
        print('Work day - 4')
        try:
          big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=4)), 'Close']
          big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=4)), 'Close']
        except:
          print('Work day - 6')
          try:
            big_data.at[(index), 'Stock Price'] = history_stock.at[str(date - timedelta(days=6)), 'Close']
            big_data.at[(index), 'Index Price'] = history_index.at[str(date - timedelta(days=6)), 'Close']
          except:
            print("Skipped")
            continue

The companies for which we are unable to find price data are removed from the dataset. The dataset is saved to csv for faster and easier access.

In [0]:
print(big_data.shape)
big_data = big_data.drop(big_data[big_data['Stock Price'].isna()].index)
print(big_data.shape)
big_data.to_csv("/content/drive/My Drive/Data Sets/fundamental_dataset_no_na.csv")

Creates a list of all companies left in the data set and sets Ticker and Report date as an index.

In [0]:
big_data.reset_index(inplace=True)
big_data.set_index('Ticker', inplace=True)
company_index = big_data.index.unique()
print(company_index)
big_data.reset_index(inplace=True)
big_data.set_index(['Ticker', 'Report Date'], inplace=True)
big_data

Calculates the percent change of the stock and market prices since the first report we have available for the current date. Calculates the difference in percentages and labels as an outperformer if the company's price has increased more than the one of the index. Otherwise, it labels the company as an underperfomer.

In [0]:
for company in company_index:
  
  reports = big_data.loc[company].index
  starting_index_value = big_data.at[(company, reports[0]), 'Index Price']
  starting_stock_value = big_data.at[(company, reports[0]), 'Stock Price']

  for report in reports:
    current_index_value = big_data.at[(company, report), 'Index Price']
    current_stock_value = big_data.at[(company, report), 'Stock Price']

    stock_percent_difference = ((current_stock_value-starting_stock_value)/starting_stock_value)* 100.0
    index_percent_difference = ((current_index_value-starting_index_value)/starting_index_value)* 100.0

    big_data.at[(company, report), 'Stock Percent Difference'] = stock_percent_difference
    big_data.at[(company, report), 'Index Percent Difference'] = index_percent_difference

    price_percent_difference = stock_percent_difference - index_percent_difference

    if price_percent_difference > 0:
      big_data.at[(company, report), 'Label'] = 1
    else:
      big_data.at[(company, report), 'Label'] = 0

Removes an unnecessary column and saves the labelled data set as a csv file.

In [0]:
big_data.drop(columns=['SimFinId'], inplace=True)
big_data.to_csv('/content/drive/My Drive/Data Sets/labelled_fundamental_data.csv')

Used to find the oldest and newest report in the labelled data set.

In [0]:
from datetime import datetime as dt
big_data = pd.read_csv('/content/drive/My Drive/Data Sets/labelled_fundamental_data.csv')
print(max(pd.to_datetime(big_data['Report Date'], format='%Y-%m-%d').dt.date))
print(min(pd.to_datetime(big_data['Report Date'], format='%Y-%m-%d').dt.date))
big_data